### Machine Learning desarrollo práctica final de módulo

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# https://ipython.readthedocs.io/en/stable/interactive/plotting.html
%matplotlib inline 

cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

In [3]:
# definimos una función para representar el resultado del ajuste
def plot_decision_boundary(X,y,h,model):
    
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    h = .05  # step size in the mesh
    
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    
    Zd = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Zd = Zd.reshape(xx.shape)
    
    Zp = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:,1] 
    Zp = Zp.reshape(xx.shape)
    
    # Error de clasificación
    ypred = model.predict(X)
    acc = accuracy_score(y,ypred)
    
    plt.figure(1, figsize=(12, 5))
    
    plt.subplot(1,2,1)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_bright)
    plt.axis([x_min, x_max, y_min, y_max])
    plt.contour(xx, yy, Zd, levels=[0], linewidths=2)
    plt.contourf(xx, yy, Zd,cmap=cm, alpha=.5)
    plt.xlabel("$x_1$", fontsize=16)
    plt.ylabel("$x_2$", fontsize=16)
    msg = 'FRONTERA DECISION\n Acc: %0.2g' % acc
    plt.title(msg)
    
    plt.subplot(1,2,2)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_bright)
    plt.axis([x_min, x_max, y_min, y_max])
    #plt.contour(xx, yy, Zp, levels=[0], linewidths=2)
    plt.contourf(xx, yy, Zp,cmap=cm, alpha=.5)
    plt.xlabel("$x_1$", fontsize=16)
    plt.ylabel("$x_2$", fontsize=16)
    msg = 'PROBABILIDAD\n Acc: %0.2g' % acc
    plt.title(msg)
    
    plt.tight_layout()
    plt.show()

In [4]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression 

# ejecute esta celda
def miRegresionLineal(x_i,y_i,x,y,degree):
    poly = PolynomialFeatures(degree)
    X_i  = poly.fit_transform(x_i.reshape(-1, 1))
    X_test = poly.fit_transform(x.reshape(-1, 1))
    lr     = LinearRegression().fit(X_i,y_i)
    
    y_hat = lr.predict(X_i)
    fw = lr.predict(X_test)
    
    error_train = np.mean(np.power(y_i-y_hat,2)) 
    error_test  = np.mean(np.power(y-fw,2)) 
    
   # print(lr.coef_)
    
    return fw, error_test, error_train

In [5]:
# Empezamos con la incorporación de los datos del fichero csv de Airbnb para generar nuestro dataset
# Y visualizamos los primeros elementos de forma transversal

airbnb_Data = pd.read_csv("./Data/airbnb-listings.csv", sep=";", decimal=".")
#airbnb_Data.head(5).T

In [6]:
#Calculamos el número de muestras y la dimensionalidad inicial.
#airbnb_Data.describe()
#airbnb_Data.isnull
#airbnb_Data.shape
N, d = airbnb_Data.shape
print(f'Numero de muestras actual es de {N} y la dimensionalidad es de {d}')

Numero de muestras actual es de 14780 y la dimensionalidad es de 89


In [7]:
#Procedemos a separar el Training y el Test, lo haremos generando dos datasets y los guardaremos
#en nuestro directorio ./Data para posteriormente cargarlos y procesarlos

from sklearn.model_selection import train_test_split

train, test = train_test_split(airbnb_Data, test_size=0.2, shuffle=True, random_state=0)
print(f'Dimensiones del dataset de training: {train.shape}')
print(f'Dimensiones del dataset de test: {test.shape}')
# Guardamos físicamente en disco
train.to_csv('./Data/Airbnb_train.csv', sep=';', decimal='.', index=False)
test.to_csv('./Data/Airbnb_test.csv', sep=';', decimal='.', index=False)

Dimensiones del dataset de training: (11824, 89)
Dimensiones del dataset de test: (2956, 89)


In [8]:
# A partir de este momento, cargaremos los datos de Training y trabajaremos con ellos para realizar la selección
# de características, los filtrados, los entrenamientos, etc.
# Y aplicaremos LAS MISMAS transformaciones que se hayan hecho en train (drop de columnas, filtros, generaciones, fill, etc) 

airbnb_train = pd.read_csv('./Data/Airbnb_train.csv', sep=';', decimal='.')

In [9]:
# A partir de este momento, cargaremos los datos de Test
# Y aplicaremos LAS MISMAS transformaciones que se hayan hecho en train (drop de columnas, filtros, generaciones, fill, etc) 

airbnb_test = pd.read_csv('./Data/Airbnb_test.csv', sep=';', decimal='.')

In [10]:
# Procedemos a eliminar rows que creemos no son necesarias de airbnb_train y lo mismo aplicaremos a airbnb_test.

airbnb_train.drop(['ID','Listing Url', 'Scrape ID', 'Last Scraped','Name','Summary','Space','Description','Experiences Offered'
                  ,'Neighborhood Overview','Notes','Transit','Access','Interaction','House Rules',
                  'Thumbnail Url','Medium Url', 'Picture Url',
                'XL Picture Url', 'Host ID', 'Host URL','Host Name','Host About','Host Thumbnail Url'
                  ,'Host Picture Url','Host Neighbourhood','Host Verifications','Street','Neighbourhood','Features',
                 'Host Location','Neighbourhood Cleansed','License','Jurisdiction Names','Host Acceptance Rate'
                 ,'Amenities','Calendar last Scraped','Calendar Updated','Host Since','State','Market'
                ,'Smart Location','Has Availability','First Review','Last Review','Weekly Price','Monthly Price',
                 'Host Response Time'],
                 axis=1,inplace=True)

In [11]:
airbnb_test.drop(['ID','Listing Url', 'Scrape ID', 'Last Scraped','Name','Summary','Space','Description','Experiences Offered'
                  ,'Neighborhood Overview','Notes','Transit','Access','Interaction','House Rules',
                  'Thumbnail Url','Medium Url', 'Picture Url',
                'XL Picture Url', 'Host ID', 'Host URL','Host Name','Host About','Host Thumbnail Url'
                  ,'Host Picture Url','Host Neighbourhood','Host Verifications','Street','Neighbourhood','Features',
                 'Host Location','Neighbourhood Cleansed','License','Jurisdiction Names','Host Acceptance Rate'
                 ,'Amenities','Calendar last Scraped','Calendar Updated','Host Since','State','Market'
                ,'Smart Location','Has Availability','First Review','Last Review','Weekly Price','Monthly Price',
                 'Host Response Time'],
                 axis=1,inplace=True)

In [16]:
print(f'Dimensiones del dataset de training: {airbnb_train.shape}')

Dimensiones del dataset de training: (11824, 41)


In [13]:
airbnb_train.head(5).T

,0,1,2,3,4
Host Response Rate,NaN,100,NaN,100,99
Host Listings Count,1,2,26,4,10
Host Total Listings Count,1,2,26,4,10
Neighbourhood Group Cleansed,Centro,Centro,Centro,Centro,Centro
City,Madrid,Madrid,Madrid,Madrid,Madrid
Zipcode,28012,28005,28013,28005,28004
Country Code,ES,ES,ES,ES,ES
Country,Spain,Spain,Spain,Spain,Spain
Latitude,40.4079,40.4104,40.4201,40.4146,40.4205
Longitude,-3.69595,-3.71092,-3.70647,-3.70712,-3.70619


In [15]:
#Verificamos el nombre de de cada columna(dimensión)
feature_names = airbnb_train.columns[1:]
feature_names

Index(['Host Listings Count', 'Host Total Listings Count',
       'Neighbourhood Group Cleansed', 'City', 'Zipcode', 'Country Code',
       'Country', 'Latitude', 'Longitude', 'Property Type', 'Room Type',
       'Accommodates', 'Bathrooms', 'Bedrooms', 'Beds', 'Bed Type',
       'Square Feet', 'Price', 'Security Deposit', 'Cleaning Fee',
       'Guests Included', 'Extra People', 'Minimum Nights', 'Maximum Nights',
       'Availability 30', 'Availability 60', 'Availability 90',
       'Availability 365', 'Number of Reviews', 'Review Scores Rating',
       'Review Scores Accuracy', 'Review Scores Cleanliness',
       'Review Scores Checkin', 'Review Scores Communication',
       'Review Scores Location', 'Review Scores Value', 'Cancellation Policy',
       'Calculated host listings count', 'Reviews per Month', 'Geolocation'],
      dtype='object')

In [17]:
# En esta celda, filtraremos las poblaciones que contengan Madrid
# El csv descargado debía contener únicamente datos pertenecientes a Madrid

#Borramos todos los valores nulo en City
airbnb_train.dropna(subset=['City'],inplace=True)

#Cargamos el Dataset con las poblaciones que contengan Madrid no teniendo en cuenta mayúsculas y minúsculas.
airbnb_train = airbnb_train[airbnb_train["City"].str.contains('Madrid', na=False, case=False) ]

#Mostramos un conteo de todos los resultados agrupados por City
airbnb_train["City"].value_counts()

Madrid                                 10552
Madrid, Comunidad de Madrid, ES           12
madrid                                     7
MADRID                                     2
Comunidad de Madrid                        1
Centro, Madrid                             1
Madrid, Comunidad de Madrid, ESPANA        1
Chueca, Madrid                             1
Delicias-Madrid                            1
Aravaca (Madrid)                           1
las matas  madrid                          1
Name: City, dtype: int64

In [18]:
# Y haremos el mismo tratamiento de filtrado de población Madrid al dataset de test
#Borramos todos los valores nulo en City
airbnb_test.dropna(subset=['City'],inplace=True)

#Cargamos el Dataset con las poblaciones que contengan Madrid no teniendo en cuenta mayúsculas y minúsculas.
airbnb_test = airbnb_test[airbnb_test["City"].str.contains('Madrid', na=False, case=False) ]

In [19]:
print('Datos airbnb antes del borrado de NaN en Price: ', airbnb_train.shape)

Datos airbnb antes del borrado de NaN en Price:  (10580, 41)


In [20]:
#Nuestro target es el precio, por lo tanto no podemos admitir registros sin precio en train
airbnb_train.dropna(subset=['Price'],inplace=True)

In [21]:
# Y lo mismo aplicamos a test
airbnb_test.dropna(subset=['Price'],inplace=True)

In [22]:
print('Datos airbnb después del borrado de NaN en Price: ', airbnb_train.shape)

Datos airbnb después del borrado de NaN en Price:  (10575, 41)
